In [1]:
from load_results import load_result_dataset
import pandas as pd
pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'

final_data = load_result_dataset(pn1, pn2)

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [7]:
df = pd.DataFrame(final_data)
df = df[df["model_size"].isin([0, ])]
# df.to_csv("./{}.csv".format("result_with_missing_values"))

In [8]:
df

,clean_acc,Linf_acc,L2_acc,L1_acc,common_acc,sum,geom,model_size,backbone,dataset,loss_function,model_type
33,0.923810,0.000000,NaN,0.161905,NaN,NaN,NaN,0,regnetx_004.pycls_in1k,uc-merced-land-use-dataset,TRADES_v2,fully convolutional
34,0.466667,0.080952,NaN,NaN,NaN,NaN,NaN,0,efficientnet-b0,uc-merced-land-use-dataset,TRADES_v2,fully convolutional
35,0.919048,0.561905,NaN,NaN,NaN,NaN,NaN,0,deit_tiny_patch16_224.fb_in1k,uc-merced-land-use-dataset,TRADES_v2,fully attention
36,0.645238,0.245238,NaN,NaN,NaN,NaN,NaN,0,mobilevit-small,uc-merced-land-use-dataset,TRADES_v2,hybrid
37,0.961905,0.000000,NaN,NaN,NaN,NaN,NaN,0,mobilenetv3_large_100.ra_in1k,uc-merced-land-use-dataset,TRADES_v2,fully convolutional
...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.046000,0.015000,NaN,0.029000,NaN,NaN,NaN,0,deit_tiny_patch16_224.fb_in1k,oxford-iiit-pet,CLASSIC_AT,fully attention
476,0.085000,0.042000,NaN,0.042000,NaN,NaN,NaN,0,mobilevit-small,oxford-iiit-pet,CLASSIC_AT,hybrid
477,0.498000,0.003000,NaN,0.102000,NaN,NaN,NaN,0,mobilenetv3_large_100.ra_in1k,oxford-iiit-pet,CLASSIC_AT,fully convolutional
478,0.734000,0.258000,NaN,0.165333,NaN,NaN,NaN,0,edgenext_small.usi_in1k,oxford-iiit-pet,CLASSIC_AT,fully convolutional


In [9]:
import pandas as pd
import yaml


def get_restart_job(row):
    is_missing = lambda col: pd.isna(row[col])

    missing = {col: is_missing(col) for col in ['clean_acc', 'Linf_acc', 'L1_acc', 'L2_acc', 'common_acc']}

    if all(missing.values()):
        return "job1_hpo.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and missing['L1_acc']:
        return "job4_test_l1.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and missing['L2_acc']:
        return "job5_test_l2.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and not missing['L2_acc'] and missing['common_acc']:
        return "job6_test_common.sh"
    else:
        return None  # means no job needs to be restarted

df['restart_from'] = df.apply(get_restart_job, axis=1)

to_restart = df[df['restart_from'].notna()][['backbone', 'dataset', 'loss_function', 'restart_from']]
to_restart.to_csv("restart_jobs.csv", index=False)
to_restart

,backbone,dataset,loss_function,restart_from
33,regnetx_004.pycls_in1k,uc-merced-land-use-dataset,TRADES_v2,job5_test_l2.sh
34,efficientnet-b0,uc-merced-land-use-dataset,TRADES_v2,job4_test_l1.sh
35,deit_tiny_patch16_224.fb_in1k,uc-merced-land-use-dataset,TRADES_v2,job4_test_l1.sh
36,mobilevit-small,uc-merced-land-use-dataset,TRADES_v2,job4_test_l1.sh
37,mobilenetv3_large_100.ra_in1k,uc-merced-land-use-dataset,TRADES_v2,job4_test_l1.sh
...,...,...,...,...
475,deit_tiny_patch16_224.fb_in1k,oxford-iiit-pet,CLASSIC_AT,job5_test_l2.sh
476,mobilevit-small,oxford-iiit-pet,CLASSIC_AT,job5_test_l2.sh
477,mobilenetv3_large_100.ra_in1k,oxford-iiit-pet,CLASSIC_AT,job5_test_l2.sh
478,edgenext_small.usi_in1k,oxford-iiit-pet,CLASSIC_AT,job5_test_l2.sh


In [10]:
import collections
collections.Counter(to_restart.restart_from)

Counter({'job5_test_l2.sh': 62, 'job4_test_l1.sh': 13, 'job1_hpo.sh': 6})

In [5]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index

# df['total_acc'] = df['clean_acc'] + df['Linf_acc']  # Sum of scores per dataset

import pandas as pd

df = pd.DataFrame(final_data)

nan_percentage = (df.isna().sum().sum() / df.size) * 100
print(f"Percentage of NaN values: {nan_percentage:.2f}%")

df = df[ df.model_size == 1 ]

grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'],
    columns='dataset',
    # dropna=False
)

# Rename column levels
grouped_df.columns.set_names(["metric", "dataset"], inplace=True)

# Swap column levels → dataset becomes level 0, metric becomes level 1
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)

# (Optional) Sort columns so that all metrics are grouped within each dataset
grouped_df = grouped_df.sort_index(axis=1, level=0)

grouped_df.to_csv("./{}.csv".format(pn1))

grouped_df

Percentage of NaN values: 13.32%


dataset                                                  caltech101            \
metric                                                       L1_acc    L2_acc   
backbone                                   loss_function                        
coatnet_0_rw_224.sw_in1k                   CLASSIC_AT      0.713333  0.750000   
                                           TRADES_v2       0.731000  0.791000   
convnext_tiny.fb_in1k                      CLASSIC_AT      0.809000  0.838000   
                                           TRADES_v2       0.822000  0.850000   
convnext_tiny.fb_in22k                     CLASSIC_AT      0.436000  0.430000   
                                           TRADES_v2       0.805000  0.842000   
convnext_tiny.fb_in22k_ft_in1k             CLASSIC_AT      0.800000  0.829333   
                                           TRADES_v2       0.836000  0.871000   
deit_small_patch16_224.fb_in1k             CLASSIC_AT      0.213333  0.229000   
                                           TRADES_v2       0.735000  0.783000   
eva02_tiny_patch14_224.mim_in22k           CLASSIC_AT      0.439000  0.490000   
                                           TRADES_v2       0.722000  0.756000   
resnet50.a1_in1k                           CLASSIC_AT      0.687000  0.677000   
                                           TRADES_v2       0.621333  0.410667   
robust_convnext_tiny                       CLASSIC_AT      0.528000  0.512000   
                                           TRADES_v2       0.586667  0.844000   
robust_deit_small_patch16_224              CLASSIC_AT           NaN       NaN   
                                           TRADES_v2       0.692000  0.834000   
robust_resnet50                            CLASSIC_AT      0.156000  0.625333   
                                           TRADES_v2       0.148000  0.611000   
swin_tiny_patch4_window7_224.ms_in1k       CLASSIC_AT      0.746000  0.789333   
                                           TRADES_v2       0.732000  0.793000   
vit_small_patch16_224.augreg_in1k          CLASSIC_AT      0.184000  0.217333   
                                           TRADES_v2       0.682000  0.711000   
vit_small_patch16_224.augreg_in21k         CLASSIC_AT      0.586000  0.648000   
                                           TRADES_v2       0.044000  0.000000   
vit_small_patch16_224.augreg_in21k_ft_in1k CLASSIC_AT      0.633333  0.706000   
                                           TRADES_v2       0.702000  0.712000   
vit_small_patch16_224.dino                 CLASSIC_AT      0.512000  0.604000   
                                           TRADES_v2       0.546000  0.552000   

dataset                                                                       \
metric                                                    Linf_acc clean_acc   
backbone                                   loss_function                       
coatnet_0_rw_224.sw_in1k                   CLASSIC_AT     0.675000  0.856000   
                                           TRADES_v2      0.734667  0.880000   
convnext_tiny.fb_in1k                      CLASSIC_AT     0.769000  0.925000   
                                           TRADES_v2      0.778000  0.929000   
convnext_tiny.fb_in22k                     CLASSIC_AT     0.380000  0.464000   
                                           TRADES_v2      0.763000  0.937000   
convnext_tiny.fb_in22k_ft_in1k             CLASSIC_AT     0.765333  0.905333   
                                           TRADES_v2      0.830000  0.950000   
deit_small_patch16_224.fb_in1k             CLASSIC_AT     0.187000  0.261000   
                                           TRADES_v2      0.705000  0.895000   
eva02_tiny_patch14_224.mim_in22k           CLASSIC_AT     0.426000  0.568000   
                                           TRADES_v2      0.678000  0.864000   
resnet50.a1_in1k                           CLASSIC_AT     0.531000  0.907000   
                                           TRA

In [6]:

# --- existing shortcuts ----------------------------------------------------
sum_scores  = grouped_df.xs('sum',  level=1, axis=1)     # (rows × 6 datasets)
geom_scores = grouped_df.xs('geom', level=1, axis=1)     # (rows × 6 datasets)

#--- 1. add the TOTAL aggregates you already computed ----------------------
grouped_df[('TOTAL', 'score_sum')]  = sum_scores.sum(axis=1)
grouped_df[('TOTAL', 'score_geom')] = geom_scores.sum(axis=1)

# --- 2. count NaNs across datasets -----------------------------------------
grouped_df[('TOTAL', 'nan_sum_cnt')]  = sum_scores.isna().sum(axis=1)
grouped_df[('TOTAL', 'nan_geom_cnt')] = geom_scores.isna().sum(axis=1)

# --- 3. rank as before ------------------------------------------------------
df_sorted = grouped_df.sort_values(('TOTAL', 'score_sum'), ascending=False)

df_sorted

dataset                                                  caltech101            \
metric                                                       L1_acc    L2_acc   
backbone                                   loss_function                        
convnext_tiny.fb_in1k                      TRADES_v2       0.822000  0.850000   
convnext_tiny.fb_in22k                     TRADES_v2       0.805000  0.842000   
robust_convnext_tiny                       TRADES_v2       0.586667  0.844000   
convnext_tiny.fb_in22k_ft_in1k             TRADES_v2       0.836000  0.871000   
convnext_tiny.fb_in1k                      CLASSIC_AT      0.809000  0.838000   
robust_deit_small_patch16_224              TRADES_v2       0.692000  0.834000   
robust_resnet50                            CLASSIC_AT      0.156000  0.625333   
swin_tiny_patch4_window7_224.ms_in1k       CLASSIC_AT      0.746000  0.789333   
robust_resnet50                            TRADES_v2       0.148000  0.611000   
swin_tiny_patch4_window7_224.ms_in1k       TRADES_v2       0.732000  0.793000   
vit_small_patch16_224.augreg_in1k          TRADES_v2       0.682000  0.711000   
convnext_tiny.fb_in22k_ft_in1k             CLASSIC_AT      0.800000  0.829333   
resnet50.a1_in1k                           TRADES_v2       0.621333  0.410667   
convnext_tiny.fb_in22k                     CLASSIC_AT      0.436000  0.430000   
deit_small_patch16_224.fb_in1k             CLASSIC_AT      0.213333  0.229000   
eva02_tiny_patch14_224.mim_in22k           TRADES_v2       0.722000  0.756000   
vit_small_patch16_224.augreg_in21k         TRADES_v2       0.044000  0.000000   
resnet50.a1_in1k                           CLASSIC_AT      0.687000  0.677000   
coatnet_0_rw_224.sw_in1k                   TRADES_v2       0.731000  0.791000   
vit_small_patch16_224.augreg_in21k_ft_in1k TRADES_v2       0.702000  0.712000   
robust_deit_small_patch16_224              CLASSIC_AT           NaN       NaN   
deit_small_patch16_224.fb_in1k             TRADES_v2       0.735000  0.783000   
robust_convnext_tiny                       CLASSIC_AT      0.528000  0.512000   
vit_small_patch16_224.dino                 CLASSIC_AT      0.512000  0.604000   
vit_small_patch16_224.augreg_in21k         CLASSIC_AT      0.586000  0.648000   
coatnet_0_rw_224.sw_in1k                   CLASSIC_AT      0.713333  0.750000   
eva02_tiny_patch14_224.mim_in22k           CLASSIC_AT      0.439000  0.490000   
vit_small_patch16_224.dino                 TRADES_v2       0.546000  0.552000   
vit_small_patch16_224.augreg_in21k_ft_in1k CLASSIC_AT      0.633333  0.706000   
vit_small_patch16_224.augreg_in1k          CLASSIC_AT      0.184000  0.217333   

dataset                                                                       \
metric                                                    Linf_acc clean_acc   
backbone                                   loss_function                       
convnext_tiny.fb_in1k                      TRADES_v2      0.778000  0.929000   
convnext_tiny.fb_in22k                     TRADES_v2      0.763000  0.937000   
robust_convnext_tiny                       TRADES_v2      0.899000  0.967000   
convnext_tiny.fb_in22k_ft_in1k             TRADES_v2      0.830000  0.950000   
convnext_tiny.fb_in1k                      CLASSIC_AT     0.769000  0.925000   
robust_deit_small_patch16_224              TRADES_v2      0.834667  0.954667   
robust_resnet50                            CLASSIC_AT     0.829000  0.957000   
swin_tiny_patch4_window7_224.ms_in1k       CLASSIC_AT     0.709000  0.884000   
robust_resnet50                            TRADES_v2      0.832000  0.958000   
swin_tiny_patch4_window7_224.ms_in1k       TRADES_v2      0.718000  0.902000   
vit_small_patch16_224.augreg_in1k          TRADES_v2      0.602667  0.872000   
convnext_tiny.fb_in22k_ft_in1k             CLASSIC_AT     0.765333  0.905333   
resnet50.a1_in1k                           TRADES_v2      0.171000  0.901000   
convnext_tiny.fb_in22k                     CLA

In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Assuming `df1` and `df_per_row_sums` already exist

# Define metrics
metrics = ['sum_L1_acc', 'sum_L2_acc', 'sum_Linf_acc', 'sum_clean_acc', 'sum_common_acc']

# Assign a unique color per observation
unique_labels = [f"{b} ({l})" for b, l in df_per_row_sums.index]
color_map = dict(zip(
    unique_labels,
    px.colors.qualitative.Plotly * (len(unique_labels) // len(px.colors.qualitative.Plotly) + 1)  # ensure enough colors
))

# Create radar plot
fig = go.Figure()

for (backbone, loss), row in df_per_row_sums.iterrows():
    label = f"{backbone} ({loss})"
    fig.add_trace(go.Scatterpolar(
        r=row.tolist() + [row.tolist()[0]],  # Close the loop
        theta=metrics + [metrics[0]],
        mode='lines',
        name=label,
        line=dict(width=2, color=color_map[label])
    ))

fig.update_layout(
    title="Radar Plot of Raw Accuracy Metrics",
    polar=dict(
        radialaxis=dict(visible=True),
        angularaxis=dict(direction="clockwise")
    ),
    showlegend=True
)

fig.show()
